In [3]:
from konlpy.tag import Okt
from gensim.models.fasttext import FastText, load_facebook_vectors, load_facebook_model
import os
import pandas as pd
import re
from gensim.test.utils import datapath

def preprocess_korean_text(text, stop_words=[]):
    okt = Okt()
    
    # 형태소 분석을 통한 토큰화
    words = okt.morphs(text, norm=True, stem=True) # 정규화 처리
    
    # 불용어 제거
    words = [word for word in words if word not in stop_words]
    
    return words

def process_csv_file(csv_path, stop_words=[]):
    new_data = []
    
    df = pd.read_csv(csv_path, encoding='utf-8')
    # if 'reviews' in df.columns:
    #     # for text in df['reviews']:
    #     #     # 추가적인 전처리 작업 (예: 특수 문자 제거, 숫자 제거 등)
    #     #     text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)
    #     #     
    #     #     # 전처리된 문장을 추가 데이터에 포함
    #     #     tokens = preprocess_korean_text(text, stop_words)
    #     #     new_data.append(tokens)
    #     for name, text in zip(df['name'], df['reviews']):
    #         # 추가적인 전처리 작업 (예: 특수 문자 제거, 숫자 제거 등)
    #         text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)
    #         
    #         # 전처리된 문장과 매장명을 추가 데이터에 포함
    #         processed_text = preprocess_korean_text(text, stop_words)
    #         processed_text.insert(0, name)
    #         new_data.append(processed_text)
    new_data = [
            [
                name,
                # clean_mark,
                # delivery_cost, least_cost, 
                # review_event, 
                # taste_star,quantity_star, delivery_star,
                *preprocess_korean_text(re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text), stop_words)
            ]
        # for name, text, clean_mark, delivery_cost, least_cost, review_event, taste_star, quantity_star, delivery_star in zip(
        #     df['name'], df['reviews'], df['cleanMark'], df['delivery_cost'], df['least_cost'], df['reviewEvent'], df['taste_star'], df['quantity_star'], df['delivery_star']
        # )
        for name, text in zip(df['name'], df['reviews'])
    ]

    return new_data

# 새로운 데이터 파일 경로
input_csv_file_name = "reviews_converted_data_tagging.csv"
csv_file_path = f'../ReviewData/{input_csv_file_name}'

# 불용어 리스트 (필요에 따라 추가)
stop_words = ['의','가', '에','들','는','잘','걍','과','도','를','으로','한','하다','!','?','<','>','(',')','[',']','|','#','.', '이','은','는','을','에','에서','로']

# CSV 파일 처리 및 전처리
new_data = process_csv_file(csv_file_path, stop_words)
print("preprocess done")
print(new_data[0:101])

preprocess done
[['요리왕-성북구청점', '오랜', '만', '완두콩', '옥수수', '들다', '짜장면', '먹다', '보다', '배달', '무척', '이나', '빠르다', '짬뽕', '국물', '서비스', '오니', '야식', '시키다', '때', '정말', '딱이다', '근데', '제', '입맛', '엔', '조금', '짜다'], ['요리왕-성북구청점', '배달', '초', '스피드', '개맛', '있다'], ['요리왕-성북구청점', '잡채', '밥', '첨', '시키다', '맛있다'], ['요리왕-성북구청점', '맛', '양', '까지', '다', '최고', '예요'], ['요리왕-성북구청점', '맛있다', '먹다'], ['요리왕-성북구청점', '맛있다', '자다', '먹다'], ['요리왕-성북구청점', '아니다', '제', '도착', '문자', '해달라다', '체크', '놧', '늘다', '시간', '넘다', '지나다', '안', '오다', '레', '문자', '혹시', '나', '해', '서문', '밖에', '보', '니깐', '음식', '오다', '이미', '음식', '춥다', '다', '식', '엇', '곱다', '차갑다', '상태', '네', '요', '문자', '하', '나', '써주다', '뭐', '어렵다', '참나', '다시다', '안', '시키다', '먹다'], ['요리왕-성북구청점', '너무', '맛있다'], ['요리왕-성북구청점', '탕수육', '맛있다', '짬뽕', '짜장', '제', '입맛', '에는', '좀', '짜다', '느껴지다', '네', '요담', '엔', '싱겁다', '요청', '드리다'], ['요리왕-성북구청점', '인분', '배달', '되다', '곳', '중', '제일', '맛있다', '여기', '서', '만', '시키다', '구', '요'], ['요리왕-성북구청점', '아침', '일찍', '인분', '배달', '돼다', '넘다', '좋다'], ['요리왕-성북구청점', '여기', '볶음밥', '직접',

In [79]:
# # 기존 pre-trained 모델 경로
# pretrained_model_path = 'wiki.ko.bin'
# 
# # 모델 로드
# # model = load_facebook_vectors(pretrained_model_path)
# model = load_facebook_model(pretrained_model_path)
# 
# # 모델 업데이트
# model.build_vocab(corpus_iterable=new_data, update=True)
# model.train(corpus_iterable=new_data, total_examples=len(new_data), epochs=model.epochs)

# 직접 만든 모델
model = FastText(sentences=new_data, window=5, min_count=10, workers=4, sg=1, epochs=100)

# 업데이트된 모델 저장
updated_model_path = f'updated_model_{input_csv_file_name.split(".")[0]}.bin'
model.save(updated_model_path)

# 저장된 모델 경로 출력
print(f"모델이 성공적으로 업데이트되었습니다. 업데이트된 모델 경로: {os.path.abspath(updated_model_path)}")

모델이 성공적으로 업데이트되었습니다. 업데이트된 모델 경로: /Users/heojw/Desktop/Workspace/univ/3-2/NaturalLanguageProcessing/TermProject/Preprocess/updated_model_reviews_name.bin


In [80]:
from gensim.models.fasttext import FastText

# 저장한 모델 불러오기
model = FastText.load(f'./{updated_model_path}')

def find_similar_words(word, topn=10):
    try:
        # 입력한 단어와 유사한 단어들을 찾기
        similar_words = model.wv.most_similar(word, topn=topn)

        # 결과 출력
        print(f"입력한 단어 '{word}'와(과) 유사한 단어들:")
        for similar_word, similarity in similar_words:
            print(f"{similar_word}: {similarity}")

    except KeyError:
        print(f"입력한 단어 '{word}'는 모델의 어휘에 없습니다.")



In [85]:
# 사용자로부터 단어 입력 받기
user_input = input("단어를 입력하세요: ")

# 유사한 단어 찾기 및 출력
find_similar_words(user_input)

입력한 단어 '교촌치킨'와(과) 유사한 단어들:
교촌: 0.7284786701202393
동대문: 0.5651504397392273
교촌치킨-동대문1호점: 0.557990312576294
콤보: 0.5523183345794678
허니: 0.5053131580352783
지: 0.48752468824386597
치킨: 0.48023173213005066
골드: 0.47006797790527344
코바: 0.46466726064682007
허니콤: 0.4635542035102844


In [4]:
from konlpy.tag import Okt
from gensim import corpora
from gensim.models import LdaModel

# 예시 리뷰 데이터 (한글)
# reviews = [
#     "잡채밥은 첨시켰는데 맛있어요!",
#     "맛과 양까지 다 최고예요",
#     "맛있게 잘먹었습니다",
#     "맛있게 잘 먹었습니다.",
#     "아니 제가 도착하면 문자해달라고 체크해놧는데한시간 넘게 지나서도 안오길레 문자가 혹시나 해서문밖에 보니깐 음식 와있네요^^이미 음식은 추워서 다 식엇고 차가운상태네요문자하나 써주는게 뭐가 어렵다고 참나다신 안시켜먹을듯 하네요",
#     "너무 맛있어요！！！！！！！！"
# ]
# 
# # konlpy를 사용하여 명사만 추출하는 전처리 함수 정의
# def preprocess(text):
#     okt = Okt()
#     nouns = okt.nouns(text)
#     return nouns
# 
# # 전처리된 텍스트 생성
# processed_reviews = [preprocess(review) for review in reviews]
processed_reviews = new_data

# Gensim을 위한 사전(dictionary)과 코퍼스(corpus) 생성
dictionary = corpora.Dictionary(processed_reviews)
corpus = [dictionary.doc2bow(text) for text in processed_reviews]

# LDA 모델 훈련
lda_model = LdaModel(corpus, num_topics=20, id2word=dictionary, passes=15)

# 토픽 출력
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.231*"배달" + 0.126*"빠르다" + 0.125*"맛있다" + 0.116*"감사하다" + 0.053*"오다"')
(1, '0.102*"밥" + 0.080*"고" + 0.068*"좋다" + 0.055*"주다" + 0.054*"새벽"')
(2, '0.210*"정말" + 0.091*"님" + 0.076*"사장" + 0.073*"맛있다" + 0.050*"짱"')
(3, '0.097*"보다" + 0.089*"먹다" + 0.067*"맛있다" + 0.055*"시키다" + 0.053*"처음"')
(4, '0.355*"너무" + 0.181*"맛있다" + 0.107*"진짜" + 0.059*"먹다" + 0.045*"사진"')
(5, '0.103*"서비스" + 0.095*"짬뽕" + 0.088*"맛" + 0.087*"먹다" + 0.083*"나다"')
(6, '0.157*"요" + 0.075*"맛" + 0.074*"나" + 0.048*"순" + 0.043*"치킨"')
(7, '0.061*"같다" + 0.038*"것" + 0.034*"없다" + 0.027*"보다" + 0.023*"되다"')
(8, '0.064*"리뷰" + 0.051*"안" + 0.046*"ㅠㅠ" + 0.032*"받다" + 0.026*"오다"')
(9, '0.077*"다" + 0.055*"아쉽다" + 0.051*"좀" + 0.051*"조금" + 0.048*"맛있다"')
(10, '0.128*"배달" + 0.114*"시간" + 0.077*"늦다" + 0.062*"오다" + 0.049*"분"')
(11, '0.165*"먹다" + 0.101*"시키다" + 0.080*"ㅎㅎ" + 0.076*"맛있다" + 0.052*"여기"')
(12, '0.215*"좋다" + 0.154*"고기" + 0.087*"있다" + 0.082*"맛" + 0.034*"튀김"')
(13, '0.210*"양" + 0.125*"많다" + 0.067*"국물" + 0.064*"맛있다" + 0.047*"비"')
(14, '0.177*"맛" + 0

In [15]:
def pandas_review_preprocessing(review : str):
    return " ".join(preprocess_korean_text(re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', review), stop_words))
    
df = pd.read_csv("../ReviewData/reviews_converted_data_tagging.csv")
df['reviews'] = df['reviews'].apply(pandas_review_preprocessing)
df.head()

KeyboardInterrupt: 

In [18]:
df = pd.read_csv("../ReviewData/reviews_converted_data_tagging.csv")
df.drop(['Unnamed: 0', 'reviewEvent'], axis=1, inplace=True)
# CSV 파일로 저장, 인덱스 제외 및 인코딩 지정
df.to_csv('../ReviewData/reviews_last.csv', index=False, encoding='utf-8-sig')